In [ ]:
import os
import numpy as np
import scipy.sparse
import scipy.io
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy import stats 
from scipy import * 
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
sc.settings.figdir = "/home/krushna/Documents/Data_integration/Experiments/DC monocytes/Figures"
sc.set_figure_params(dpi_save=200)
sc.set_figure_params(figsize=(6.4, 4.8))

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch"
                 
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster"
                    }    
# emb_folder = "/home/krushna/Documents/Data_integration/Ajita_embeddings/final/"

In [ ]:
dataset = 'Immune_Human'
method = 'scvi'
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]
adata = sc.read_h5ad("/home/krushna/Documents/Data_integration/Othermethods/scvi/scvi-Immune_Human.h5ad")

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=False,
        embed='X_scVI',
        si_metric='euclidean',
        pcr_=False,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=False,
        kBET_=False,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=False,
        type_=None,
        verbose=True,
)
results

In [ ]:
sc.pp.neighbors(adata, use_rep='X_scVI')
sc.tl.louvain(adata, resolution=0.23692631578947368,key_added='cluster')

In [ ]:
# sc.pp.neighbors(adata, use_rep='X_scVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=cell_type, frameon=False,save = '_'+method+'_cell_type.png')
sc.pl.umap(adata, color=batch, frameon=False,save='_'+method+'_batch.png')
sc.pl.umap(adata, color='cluster', frameon=False,save='_'+method+'_opt_cluster.png')

In [ ]:
sc.pl.umap(adata, color=['CLEC9A','CD1C','VCAN'], frameon=False,save='_'+method+'_DC123_markers.png')

In [ ]:
sc.pl.umap(adata, color='cluster', frameon=False,save='_'+method+'_opt_cluster.png',title='scVI louvain clusters')

# view markers on all data DC

In [ ]:
#1
sc.pl.umap(adata, color=["CLEC9A","HLA-DPA1","CADM1","CAMK2D",cell_type], frameon=False)#"C1ORF54"

# ["CD1C","FCER1A","CLEC10A","ADAM8","CD1D"]
# ["S100A9","S100A8","VCAN","LYZ","ANXA1"]
# ["FCGR3A","FTL","SERPINA1","LST1","AIF1"]
# ["AXL","PPP1R14A","SIGLEC6","CD22","DAB2"]
# ["GZMB","IGJ","AK128525","SERPINF1","ITM2C"]

In [ ]:
#2
sc.pl.umap(adata, color=["CD1C","FCER1A","CLEC10A","ADAM8","CD1D",cell_type], frameon=False)

In [ ]:
#3
sc.pl.umap(adata, color=["S100A9","S100A8","VCAN","LYZ","ANXA1",cell_type], frameon=False)

In [ ]:
#4
sc.pl.umap(adata, color=["FCGR3A","FTL","SERPINA1","LST1","AIF1",cell_type], frameon=False)

In [ ]:
#5 novel cluster
sc.pl.umap(adata, color=["AXL","PPP1R14A","SIGLEC6","CD22","DAB2",cell_type], frameon=False)

In [ ]:
#6
sc.pl.umap(adata, color= ["GZMB","SERPINF1","ITM2C",cell_type], frameon=False)#"IGJ","AK128525",

# subset DC subsetypes

In [ ]:
adata_sub=adata[adata.obs[cell_type].isin(["Monocyte-derived dendritic cells","Plasmacytoid dendritic cells"])]

In [ ]:
sc.pp.neighbors(adata_sub, use_rep='X_scVI')

## check resolution

In [ ]:
for i in [0.1,0.2,0.3,0.4,0.5,1]:
    print("resolution :",i)
    sc.tl.louvain(adata_sub, resolution=i)
    sc.pl.umap(adata_sub,color=['louvain',cell_type],frameon=False)

## view marker genes of clustes

In [ ]:
sc.tl.louvain(adata_sub, resolution=0.4)
sc.pl.umap(adata_sub,color=['louvain'],frameon = False,save='_'+method+'_dc_sub_opt_res.png')

In [ ]:
#1
print('DC1')
sc.pl.umap(adata_sub, color=["CLEC9A","HLA-DPA1","CADM1","CAMK2D","louvain",cell_type], frameon=False)#"C1ORF54"
print('DC2')#2
sc.pl.umap(adata_sub, color=["CD1C","FCER1A","CLEC10A","ADAM8","CD1D","louvain",cell_type], frameon=False)
print('DC3')#3
sc.pl.umap(adata_sub, color=["S100A9","S100A8","VCAN","LYZ","ANXA1","louvain",cell_type], frameon=False)
print('DC4')#4
sc.pl.umap(adata_sub, color=["FCGR3A","FTL","SERPINA1","LST1","AIF1","louvain",cell_type], frameon=False)
print('DC6')#6
sc.pl.umap(adata_sub, color= ["GZMB","SERPINF1","ITM2C",cell_type], frameon=False)#"IGJ","AK128525",
# ["CD1C","FCER1A","CLEC10A","ADAM8","CD1D"]
# ["S100A9","S100A8","VCAN","LYZ","ANXA1"]
# ["FCGR3A","FTL","SERPINA1","LST1","AIF1"]
# ["AXL","PPP1R14A","SIGLEC6","CD22","DAB2"]
# ["GZMB","IGJ","AK128525","SERPINF1","ITM2C"]

In [ ]:
#5 novel cluster
sc.pl.umap(adata_sub, color=["AXL","PPP1R14A","SIGLEC6","CD22","DAB2","louvain",cell_type], frameon=False)

In [ ]:
markers_all = {'DC1':["CLEC9A","HLA-DPA1","CADM1","CAMK2D"],
           'DC2':["CD1C","FCER1A","CLEC10A","ADAM8","CD1D"],
           'DC3':["S100A9","S100A8","VCAN","LYZ","ANXA1"],
           'DC4':["FCGR3A","FTL","SERPINA1","LST1","AIF1"],
           'DC5':["AXL","PPP1R14A","SIGLEC6","CD22","DAB2","CD22"],
           'DC6':["GZMB","SERPINF1","ITM2C"]
          }
markers = {'DC1':["CLEC9A","CAMK2D"],
           'DC2':["CD1C","CD1D"],
           'DC3':["S100A8","VCAN"],
           'DC4':["FCGR3A","SERPINA1"],
           'DC5':["AXL","PPP1R14A"],
           'DC6':["GZMB","SERPINF1"]
           
}
sc.pl.stacked_violin(adata_sub ,markers, groupby='louvain',categories_order=['2','0','4','3','1'],figsize=(15,5),cmap='YlGn',save= "_"+dataset+'_'+method+"_DC violin.png")

# violin plots

In [ ]:
print("DC!")
sc.pl.violin(adata_sub, ["CLEC9A","CAMK2D"], groupby='louvain',save = '_'+method+'_immune_human_DC1.png')#,frameon=False) #"HLA-DPA1","CADM1",
print('DC2')#2
sc.pl.violin(adata_sub, ["CD1C","CD1D"], groupby='louvain',save = '_'+method+'_immune_human_DC2.png')
print('DC3')#3
sc.pl.violin(adata_sub, ["S100A8","VCAN"], groupby='louvain',save = '_'+method+'_immune_human_DC3.png')
print('DC4')#4
sc.pl.violin(adata_sub, ["FCGR3A","SERPINA1"], groupby='louvain',save = '_'+method+'_immune_human_DC4.png')
print("DC5")
sc.pl.violin(adata_sub, ["AXL","PPP1R14A","CD22"], groupby='louvain',save = '_'+method+'_immune_human_DC5.png')
print('DC6')#6
sc.pl.violin(adata_sub, ["GZMB","SERPINF1"], groupby='louvain',save = '_'+method+'_immune_human_DC6.png')

# Monocytes sub clustering

In [ ]:
adata_sub=adata[adata.obs[cell_type].isin(["CD16+ Monocytes","CD14+ Monocytes",'Monocyte progenitors'])]

In [ ]:
sc.pp.neighbors(adata_sub, use_rep='X_scVI')

In [ ]:
for i in [0.1,0.2,0.3,0.4,0.5,1]:
    print("resolution :",i)
    sc.tl.louvain(adata_sub, resolution=i)
    sc.pl.umap(adata_sub,color=['louvain',cell_type])

In [ ]:
sc.tl.louvain(adata_sub, resolution=0.2)

In [ ]:
print('mono1')
sc.pl.umap(adata_sub, color=["CD14","VCAN","S100A8","S100A9","FCN1","louvain",cell_type], frameon=False)
print('mono2')
sc.pl.umap(adata_sub, color=["LAIR2","ASAH1","APOBEC3A","TSPAN14","LIPA","louvain",cell_type], frameon=False)

In [ ]:
print('novel clusters')
print('mono3')
sc.pl.umap(adata_sub, color=["G0S2","NAMPT","NEAT1","CSF3R","louvain",cell_type], frameon=False)#,"AL137655"
print('mono4')
sc.pl.umap(adata_sub, color=["PRF1","GNLY","KLRC4-KLRK1","CTSW","louvain",cell_type], frameon=False)#"TCRBV3S1"
